In [1]:
import sys
import os
sys.path.append("../")
sys.path.append("../..")

from KETIPrePartialDataPreprocessing import makeNaNImputationTest

inputType ='influx' # or file
refine_param = {'removeDuplication':True, 'staticFrequency':True}
outlier_param= {'certainOutlierToNaN':True, 'uncertainOutlierToNaN':True, 'data_type':'air'}
imputation_param ={
"imputation_method":[
    {"min":0,"max":2,"method":"linear"}],"totalNanLimit":30}

# mean, median X
# ffill~polynomial_interpolate=> 각 구간마다 딱 고만큼만 Fill

input_data = makeNaNImputationTest.inputControl(inputType)
from KETIPrePartialDataPreprocessing import data_preprocessing
import numpy as np
data = input_data.replace(4.0, np.nan)
data = data[:1000]

In [2]:
from KETIPrePartialDataPreprocessing import data_preprocessing

MDP = data_preprocessing.DataPreprocessing()
refined_data = MDP.get_refinedData(data, refine_param)
datawithMoreCertainNaN, datawithMoreUnCertainNaN = MDP.get_outlierToNaNData(refined_data, outlier_param)
imputed_data = MDP.get_imputedData(datawithMoreUnCertainNaN, imputation_param)

after removeDuplication 1000
after make_static_frequency 1276
NaN Ratio in_ciai 21.630094043887148
(Timestamp('2021-02-04 16:53:00+0000', tz='UTC'), 4)
380
4
                           in_ciai
time                              
2021-02-04 16:53:00+00:00      NaN
2021-02-04 16:54:00+00:00      NaN
2021-02-04 16:55:00+00:00      NaN
2021-02-04 16:56:00+00:00      NaN
2021-02-04 16:57:00+00:00     96.0
2021-02-04 16:58:00+00:00     96.0
2021-02-04 16:59:00+00:00     96.0
(Timestamp('2021-02-05 01:59:00+0000', tz='UTC'), 6)
926
6
                           in_ciai
time                              
2021-02-05 01:59:00+00:00      NaN
2021-02-05 02:00:00+00:00      NaN
2021-02-05 02:01:00+00:00      NaN
2021-02-05 02:02:00+00:00      NaN
2021-02-05 02:03:00+00:00      NaN
2021-02-05 02:04:00+00:00      NaN
2021-02-05 02:05:00+00:00     94.0
2021-02-05 02:06:00+00:00     94.0
2021-02-05 02:07:00+00:00     94.0
NaN Ratio in_cici 21.630094043887148
(Timestamp('2021-02-04 16:53:00+0000', tz='UTC

../KETIPrePartialDataPreprocessing/data_imputation/nanMasking.py:26: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[indexLocation+tresh:indexLocation+consecutiveNum, column_name] = np.nan
../KETIPrePartialDataPreprocessing/data_imputation/nanMasking.py:26: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[indexLocation+tresh:indexLocation+consecutiveNum, column_name] = np.nan
../KETIPrePartialDataPreprocessing/data_imputation/nanMasking.py:26: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[indexLocation+tresh:indexLocation+consecutiveNum, column_name] = np.nan
../KETIPrePartialDataPreprocess

In [ ]:
def getConsecutiveNaNInfoOverThresh(data, column_name, thresh):
    # column data: data with one column
    # thresh: Threshold value to select specific ranges
    
    a = data.index
    b = data[column_name].values
    idx0 = np.flatnonzero(np.r_[True, np.diff(np.isnan(b))!=0,True])
    count = np.diff(idx0)
    idx = idx0[:-1]
    valid_mask = (count>=thresh) & np.isnan(b[idx])
    out_idx = idx[valid_mask]
    out_num = a[out_idx]
    out_count = count[valid_mask]
    out = zip(out_num, out_count)
    return out

def setNaNSpecificDuration(data, column_name, NaNInfoOverThresh, tresh):
    for NaNInfoOverThreshitem in NaNInfoOverThresh:
        indexLocation = data.index.get_loc(NaNInfoOverThreshitem[0])
        consecutiveNum= NaNInfoOverThreshitem[1]
        data.loc[indexLocation+thresh:indexLocation+consecutiveNum, column_name] = np.nan
    return data


In [24]:
a = data.index
b = data.values

b

array([53., 67., 69., ..., 95., 95., 95.])

In [ ]:
column_name='in_ciai'
data =datawithMoreCertainNaN  
thresh = 3

NaNInfoOverThresh = list(getConsecutiveNaNInfoOverThresh(data, column_name, thresh))
DataWithMaskedNaN = setNaNSpecificDuration(data, column_name, NaNInfoOverThresh, thresh)

In [85]:
NaNInfoOverThresh[0][0]

Timestamp('2021-02-04 15:45:00+0000', tz='UTC')

In [89]:
data.loc[indexLocation+thresh:indexLocation+consecutiveNum, column_name] = 0

/Users/moonjaewon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  """Entry point for launching an IPython kernel.


In [91]:
data[indexLocation+thresh:indexLocation+consecutiveNum]

,in_ciai,in_cici,in_cici_co2,in_cici_humi,in_cici_noise,in_cici_pm10,in_cici_pm25,in_cici_temp,in_cici_voc,in_co2,in_humi,in_noise,in_pm01,in_pm01_raw,in_pm10,in_pm10_raw,in_pm25,in_pm25_raw,in_temp,in_voc
time,,,,,,,,,,,,,,,,,,,,
2021-02-04 15:48:00+00:00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
count[valid_mask]

array([ 4,  3,  3,  8,  4,  3,  3,  4,  4,  3,  3, 10,  4,  4,  4])

In [ ]:
flat